<a href="https://colab.research.google.com/github/AllaudinAli/Machine-Learning/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


**Part A**


---



In [387]:
import numpy as np
from matplotlib.image import imread
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import re

Adding dataset

In [388]:
df = pd.read_csv('BordeauxWines.csv')
print(df.head())

                                        Name  Year  Score Price  BLOOD ORANGE  \
0          ChÃ¢teau Croix Figeac St.-Emilion  2008     84  $20              0   
1              ChÃ¢teau Fonroque St.-Emilion  2008     84  $29              0   
2  ChÃ¢teau Grand Bertin de St.-Clair MÃ©doc  2008     84   $NA             0   
3      ChÃ¢teau Lion Beaulieu Bordeaux White  2008     84   $NA             0   
4           ChÃ¢teau Marsau CÃ´tes de Francs  2008     84  $20              0   

   CITRUS  CITRUS PEEL  CITRUS ZEST  CLEMENTINE  LIME  ...  SKUNK  \
0       0            0            0           0     0  ...      0   
1       0            0            0           0     0  ...      0   
2       0            0            0           0     0  ...      0   
3       0            0            0           0     0  ...      0   
4       0            0            0           0     0  ...      0   

   SULFUR DIOXIDE  WET WOOL,WET DOG  ACETIC ACID  ETHANOL  ETHYL ACETATE  \
0               0     

Omitting two variables from the dataset
Justifying:
Given dataset has the large number of features around 980, it's reasonable to select a subset of features that are likely to be most informative for predicting wine quality.
**I have Droped 'Name' and 'Price' features**

---


Because 'Name' does not effect the dataset what so ever and 'Price' feature is missing alot of data and have excessive outliers.

In [389]:
df = df.drop(['Name', 'Price'], axis=1)

Replacing Score with Class Labels

In [390]:
conditions = [
    (df['Score'] >= 90),
    (df['Score'] >= 75) & (df['Score'] <= 89),
    (df['Score'] <= 74)
]
labels = ['high', 'medium', 'low']
df['Class'] = pd.Series(np.select(conditions, labels))

df = df.drop('Score', axis=1)

# Display the updated dataset
print(df.head())


   Year  BLOOD ORANGE  CITRUS  CITRUS PEEL  CITRUS ZEST  CLEMENTINE  LIME  \
0  2008             0       0            0            0           0     0   
1  2008             0       0            0            0           0     0   
2  2008             0       0            0            0           0     0   
3  2008             0       0            0            0           0     0   
4  2008             0       0            0            0           0     0   

   GRAPEFRUIT  GRAPEFRUIT PEEL  ORANGE  ...  SULFUR DIOXIDE  WET WOOL,WET DOG  \
0           0                0       0  ...               0                 0   
1           0                0       0  ...               0                 0   
2           0                0       0  ...               0                 0   
3           0                0       0  ...               0                 0   
4           0                0       0  ...               0                 0   

   ACETIC ACID  ETHANOL  ETHYL ACETATE  ALCOHOL  F

Spliting data into Train and Test

In [391]:
X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

SVM Classification

In [392]:
svm_classifier = SVC()

# Fit the classifier on the training data
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

Calculating Accuracy

In [393]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100)

Accuracy: 71.3588850174216


Confusion Matrix and (FPR)

In [394]:
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_mat)

# Calculate False Positive Rate (FPR) for each class
num_classes = len(labels)
fpr = {}
for i in range(num_classes):
    class_label = labels[i]
    fp = sum(confusion_mat[:, i]) - confusion_mat[i, i]
    tn = sum(sum(confusion_mat)) - sum(confusion_mat[i, :]) - sum(confusion_mat[:, i]) + confusion_mat[i, i]
    fpr[class_label] = fp / (fp + tn)

# Calculate overall accuracy of the model
overall_accuracy = accuracy_score(y_test, y_pred)

# Display False Positive Rate (FPR) for each class and overall accuracy
for class_label, false_positive_rate in fpr.items():
    print(f"False Positive Rate ({class_label}): {false_positive_rate}")
print("Overall Accuracy:", overall_accuracy*100)

Confusion Matrix:
[[   0    0  819]
 [   0    0    3]
 [   0    0 2048]]
False Positive Rate (high): 0.0
False Positive Rate (medium): 0.0
False Positive Rate (low): 1.0
Overall Accuracy: 71.3588850174216




---


**Part B**


---



Creating PCA Instance

In [396]:
# Create an instance of PCA with the desired number of components
pca = PCA(n_components=15)  # Example with 15 components

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

Creating StandardScaler and Fitting Scaler on Training data

In [397]:
# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform the data
X_train_scaled = scaler.fit_transform(X_train_pca)
X_test_scaled = scaler.transform(X_test_pca)

SVM Classification

In [398]:
svm = SVC()
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_test_scaled)

Confusion Matrix / Accuracy / (FPR)

In [399]:
conf_matrix = confusion_matrix(y_test, y_pred)

# Compute the accuracy
accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)

# Compute the False Positive Rate (FPR) for each class
fpr_high = conf_matrix[0, 1:] / np.sum(conf_matrix[0, 1:])
fpr_medium = conf_matrix[1, [0, 2]] / np.sum(conf_matrix[1, [0, 2]])
fpr_low = conf_matrix[2, :-1] / np.sum(conf_matrix[2, :-1])


**Step 5**

---


The Time taken for SVM Classification using raw data is around 2mins, while the time taken with PCA is around 6sec. This is because

*   Increasing the number of components in PCA can improve accuracy by retaining more information from the original data.
*   PCA simplifies the data by projecting it onto a lower-dimensional subspace, making the classification process more efficient. 

Therefore, PCA offers faster classification times and potentially improved accuracy by capturing the most important features.